In [1]:
!pip install datasets
!pip install transformers
!pip install sentencepiece
!pip install -U huggingface_hub

  Using cached huggingface_hub-0.19.4-py3-none-any.whl.metadata (14 kB)
Using cached huggingface_hub-0.19.4-py3-none-any.whl (311 kB)
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.16.4
    Uninstalling huggingface-hub-0.16.4:
      Successfully uninstalled huggingface-hub-0.16.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
unbabel-comet 2.2.0 requires huggingface-hub<0.17.0,>=0.16.0, but you have huggingface-hub 0.19.4 which is incompatible.


In [2]:
!pip install nltk
!pip install unbabel-comet

  Using cached huggingface_hub-0.16.4-py3-none-any.whl.metadata (12 kB)
Using cached huggingface_hub-0.16.4-py3-none-any.whl (268 kB)
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.19.4
    Uninstalling huggingface-hub-0.19.4:
      Successfully uninstalled huggingface-hub-0.19.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datasets 2.15.0 requires huggingface-hub>=0.18.0, but you have huggingface-hub 0.16.4 which is incompatible.


In [3]:
# !pip3 install git+https://github.com/Unbabel/COMET.git

In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.autograd import Variable
from torch.nn import Softmax

from typing import List, Optional, Tuple, Union, Dict, Any

from datasets import load_dataset, Dataset, DatasetDict, load_metric, load_from_disk
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from transformers import M2M100ForConditionalGeneration, M2M100Tokenizer, AutoTokenizer, Seq2SeqTrainingArguments, Seq2SeqTrainer, DataCollatorForSeq2Seq, EarlyStoppingCallback
from transformers import PreTrainedModel, TrainingArguments

from nltk.translate.bleu_score import sentence_bleu

import pandas as pd
import numpy as np

import random
import math
import time
from tqdm import tqdm
import os
import json

/home/phonnego/miniconda3/envs/nlp_env/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
seed = 42
torch.manual_seed(seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
_numpy_rng = np.random.default_rng(seed)
random.seed(seed)
np.random.seed(seed)
torch.use_deterministic_algorithms(False)
os.environ['PYTHONHASHSEED'] = str(seed)

In [6]:
device = torch.device("cuda:2" if torch.cuda.is_available() else "cpu")

# Load Model

In [7]:
model_name = 'm2m100_418M'
experiment = 'grow-1-monolingual-ha-en'
dataset_hub_name = 'Parikshith/monolingual-ha'
translated_data_hub_name = 'grow-1-monolingual-ha-en'

In [8]:
# model = M2M100ForConditionalGeneration.from_pretrained(f"facebook/{model_name}")
model = M2M100ForConditionalGeneration.from_pretrained("Parikshith/ha-en-finetune-og")
model = model.to(device)
tokenizer = M2M100Tokenizer.from_pretrained(f"facebook/{model_name}")

# Load Data

In [9]:
src_lang = 'ha'
tgt_lang = 'en'
tokenizer.src_lang = "ha"
tokenizer.tgt_lang = "en"

In [10]:
dataset = load_dataset(dataset_hub_name, split="one_million")

Generating one_million split: 100%|██████████| 1000000/1000000 [00:00<00:00, 2263986.28 examples/s]


In [11]:
first_100K = dataset.select(range(100000))

# Grow Step

In [12]:
def generate_text_and_add_to_dataset(dataset, model, tokenizer):
    # Define a function to generate text using the model
    def generate_text(prompt):
        input_ids = tokenizer(prompt, return_tensors="pt", truncation=True, padding="max_length", max_length=256).input_ids.to(device)
        with torch.no_grad():
          output_ids = model.generate(input_ids, forced_bos_token_id=tokenizer.get_lang_id("en"))
          generated_text = [tokenizer.decode(output, skip_special_tokens=True) for output in output_ids]
        return generated_text

    # Add a new field to the dataset with generated text
    def add_generated_text(example):
      example["generated_text"] = generate_text(example["ha"])
      return example

    # Apply the function to each example in the dataset
    dataset = dataset.map(add_generated_text)

    return dataset

In [13]:
translated_dataset = generate_text_and_add_to_dataset(first_100K, model, tokenizer)

Map: 100%|██████████| 100000/100000 [14:27:21<00:00,  1.92 examples/s]  


In [15]:
translated_dataset

Dataset({
    features: ['ha', 'generated_text'],
    num_rows: 100000
})

In [14]:
translated_dataset.save_to_disk('/home/phonnego/lrl_capstone/Reinforcement-Learning/new_monolingual_generations/first_100k')

Saving the dataset (1/1 shards): 100%|██████████| 100000/100000 [00:00<00:00, 2080002.38 examples/s]


In [16]:
translated_dataset_1 = translated_dataset.map(lambda example: {'generated_text': example['generated_text'][0] if isinstance(example['generated_text'], list) else example['generated_text']})

Map: 100%|██████████| 100000/100000 [00:05<00:00, 19189.40 examples/s]


In [18]:
translated_dataset[0]

{'ha': 'Babu shakka, Kevin Federline ya kasance dan wasan rawa a shirin Timberlake.',
 'generated_text': ['There is no doubt that Kevin Federline played a role in the Timberlake program.']}

In [19]:
translated_dataset_1[0]

{'ha': 'Babu shakka, Kevin Federline ya kasance dan wasan rawa a shirin Timberlake.',
 'generated_text': 'There is no doubt that Kevin Federline played a role in the Timberlake program.'}

In [20]:
translated_dataset_1.save_to_disk('/home/phonnego/lrl_capstone/Reinforcement-Learning/new_monolingual_generations_1/first_100k')

Saving the dataset (1/1 shards): 100%|██████████| 100000/100000 [00:00<00:00, 1235836.06 examples/s]


In [13]:
translated_dataset.push_to_hub(translated_data_hub_name)

Uploading the dataset shards: 100%|██████████| 1/1 [00:02<00:00,  2.64s/it]


In [41]:
translated_dataset = translated_dataset.map(lambda example: {'generated_text': example['generated_text'][0] if isinstance(example['generated_text'], list) else example['generated_text']})

In [42]:
translated_dataset.push_to_hub(translated_data_hub_name)

Uploading the dataset shards: 100%|██████████| 1/1 [00:00<00:00,  1.70it/s]


In [4]:
from comet import download_model, load_from_checkpoint

model_path = download_model("Unbabel/wmt20-comet-qe-da")
model = load_from_checkpoint(model_path)

def score_samples(dataset):
  dataset = dataset.rename_column("ha", "src")
  dataset = dataset.rename_column("generated_text", "mt")
  model_output = model.predict(dataset, batch_size=8, gpus=1)
  dataset = dataset.add_column("score", model_output['scores'])
  return dataset

Fetching 5 files: 100%|██████████| 5/5 [00:00<00:00, 25827.00it/s]
Lightning automatically upgraded your loaded checkpoint from v1.3.5 to v2.1.2. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint ../../.cache/huggingface/hub/models--Unbabel--wmt20-comet-qe-da/snapshots/2e7ffc84fb67d99cf92506611766463bb9230cfb/checkpoints/model.ckpt`
Encoder model frozen.
/home/phonnego/miniconda3/envs/nlp_env/lib/python3.8/site-packages/pytorch_lightning/core/saving.py:177: Found keys that are not in the model state dict but in the checkpoint: ['encoder.model.embeddings.position_ids']


In [7]:
dataset_name = 'Parikshith/grow-1-monolingual-ha-en'
dataset_ = load_dataset(dataset_name, split="small")

In [6]:
translated_dataset_scored = score_samples(dataset_)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA GeForce RTX 3090') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3,4,5]
Predicting DataLoader 0: 100%|██████████| 12500/12500 [10:26<00:00, 19.95it/s]


In [7]:
translated_dataset_scored

Dataset({
    features: ['src', 'mt', 'score'],
    num_rows: 100000
})

In [8]:
translated_data_hub = 'grow-1-monolingual-ha-en-comet'
translated_dataset_scored.push_to_hub(translated_data_hub)

Uploading the dataset shards: 100%|██████████| 1/1 [00:03<00:00,  3.09s/it]


In [12]:
from comet import download_model, load_from_checkpoint

# model_path = download_model("Unbabel/wmt20-comet-qe-da")
model = load_from_checkpoint('/home/phonnego/lrl_capstone/Reinforcement-Learning/comet/wmt21-comet-qe-da/checkpoints/model.ckpt')

def score_samples(dataset):
  dataset = dataset.rename_column("ha", "src")
  dataset = dataset.rename_column("generated_text", "mt")
  model_output = model.predict(dataset, batch_size=8, gpus=1)
  dataset = dataset.add_column("score", model_output['scores'])
  return dataset

Lightning automatically upgraded your loaded checkpoint from v1.3.5 to v2.1.2. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint comet/wmt21-comet-qe-da/checkpoints/model.ckpt`
Encoder model frozen.
/home/phonnego/miniconda3/envs/nlp_env/lib/python3.8/site-packages/pytorch_lightning/core/saving.py:177: Found keys that are not in the model state dict but in the checkpoint: ['encoder.model.embeddings.position_ids']


In [13]:
translated_dataset_scored = score_samples(dataset_)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA GeForce RTX 3090') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3,4,5]
Predicting DataLoader 0:   3%|▎         | 324/12500 [00:15<09:52, 20.55it/s]